# Lab 5: GenAIOps - AgentCore Runtime with Langfuse Observability

## Overview

This lab explores deploying AI agents to **Amazon Bedrock AgentCore Runtime** and integrating **Langfuse** as an alternative observability solution. AgentCore Runtime provides a serverless platform for hosting AI agents at scale, while Langfuse offers specialized observability for LLM applications.

You'll learn how to deploy agents with different observability configurations, demonstrating the flexibility of AgentCore Runtime in supporting various monitoring approaches for production GenAI applications.

**What you'll learn in this lab:**
1. Deploy agents to AgentCore Runtime with AgentCore observability
2. Configure Langfuse integration as an alternative observability solution
3. Deploy agents with custom tools and Langfuse tracing
4. Deploy agents with MCP tools and comprehensive observability
5. Compare different observability approaches for production considerations

**Observability Approaches:**
- `lab5_agent.py`: AgentCore default observability (AgentCore observability)
- `lab5_agent_tools.py`: Custom tools with Langfuse tracing
- `lab5_agent_mcp.py`: MCP tools with Langfuse tracing

## Prerequisites

- Python 3.10+
- Amazon Bedrock AgentCore SDK
- Required environment variables for Langfuse
- Enable Model Invocation logging for AgentCore Observability
- Enable Transaction Search for AgentCore Observability

> If you haven't selected the kernel, please click on the "Select Kernel" button at the upper right corner, select Python Environments and choose ".venv (Python 3.9.20) .venv/bin/python Recommended".

> To execute each notebook cell, press Shift + Enter.

> ℹ️ You can **skip these prerequisite steps** if you're in an instructor-led workshop using temporary accounts provided by AWS

### Add environment variables for Langfuse
1. create a copy of `genai-ml-platform-examples/integration/genaiops-langfuse-on-aws/.env.example`
2. Rename it to `.env`
3. Add in actual values for Langfuse host, secret and public key

### Enable Model Invocation Logging
1. Navigate to 'Cloudwatch', click on 'Log groups', and copy the name of the log group prefixed with `/aws/codebuild/CodeBuildProject`
    <div style="text-align:left">
        <img src="images/agentcore_obs_cloudwatch_loggroup.png" width="100%"/>
    </div>
2. Navigate to 'Amazon Bedrock' in the AWS console.
3. Under 'Configure and learn', click on 'Settings'. Follow the set up as shown below.
    - Paste in the log group name from Step 1 in the  'Log group name' field.
    - Under 'Service role', Select 'arn:aws:iam:....:role/WSParticipantRole
    <div style="text-align:left">
        <img src="images/agentcore_obs_model_invocation.png" width="100%"/>
    </div>

### Enable Transaction Search in the console

The following procedure describes how to enable Transaction Search in the console.
To enable Transaction Search in the CloudWatch console
1. Open the CloudWatch console at https://console.aws.amazon.com/cloudwatch/
2. From the navigation pane, under Application Signals, choose Transaction Search.
3. Choose Enable Transaction Search.
4. Select the box to ingest spans as structured logs, and enter a percentage of spans to be indexed. You can index spans at 1% for free and change the percentage later based on your requirements.



In [ ]:
# Install required dependencies
!uv pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import json
import os
import uuid
from pathlib import Path

import boto3
from bedrock_agentcore_starter_toolkit import Runtime
from dotenv import load_dotenv
from IPython.display import Markdown, display


load_dotenv("../.env")

In [ ]:
# Get region
region = os.environ.get("AWS_DEFAULT_REGION") or os.environ.get("AWS_REGION")
if region:
    print(f"✅ Using region from environment: {region}")
else:
    # Default fallback
    region = "us-west-2"
    print(f"🔄 Using default region: {region}")


# Initialize boto3 clients with explicit region
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)

# Initialize AgentCore runtime
agentcore_runtime = Runtime()

In [ ]:
def invoke_agent(agent_arn, prompt):
    """Invoke the agent with the given prompt."""
    try:
        response = data_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            runtimeSessionId=str(uuid.uuid4()),
            payload=json.dumps({"prompt": prompt}).encode(),
        )
        return json.loads(response["response"].read().decode("utf-8"))
    except Exception as e:
        return f"Error invoking agent: {e}"

## Create, Deploy, and Invoke the First Agent
This section will create the first agent file (`lab5_agent.py`), deploy it to AgentCore Runtime, and invoke it remotely.

### Agent 1: Built-in Tool Agent (calculator)
This agent demonstrates AgentCore Runtime's default observability mode using the built-in `calculator` tool from Strands. When deployed to AgentCore Runtime, observability is automatically enabled through AWS OpenTelemetry instrumentation, providing comprehensive tracing to Amazon CloudWatch without requiring external tools like Langfuse.

**AgentCore Observability on Amazon CloudWatch:**
- **Automatic instrumentation**: AgentCore Runtime automatically instruments your agent with OpenTelemetry
- **CloudWatch integration**: All traces, metrics, and logs are sent to Amazon CloudWatch - GenAI Observability
- **Agent lifecycle tracking**: Monitor agent invocations, tool usage, and response times
- **No additional setup**: Works out-of-the-box without external observability platforms

**Key features:**
- Uses the `calculator` tool for mathematical operations
- Deployed as a BedrockAgentCoreApp entrypoint with default observability
- Traces automatically appear in CloudWatch for monitoring and debugging

#### Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |calculator                                             |
|Custom tools created|                                |
|MCP Tools           |          |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |AgentCore Observability                               |
</div>
<br style="clear: both;"/>


This demonstrates the simplest observability setup - AgentCore Observability.

In [ ]:
%%writefile lab5_agent.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands_tools import calculator

app = BedrockAgentCoreApp()
MODEL = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"


@app.entrypoint
def lab5_agent(payload):
    user_input = payload.get("prompt")
    print("LAB5: User input:", user_input)

    agent = Agent(
        system_prompt="You're a helpful assistant. You can do simple math calculation.",
        tools=[calculator],
        model=MODEL,
        name="lab5-agentcore-agent",
    )

    response = agent(user_input)
    response_text = response.message["content"][0]["text"]

    print(f"LAB5: Response preview: {response_text[:100]}...")

    return response_text


if __name__ == "__main__":
    app.run()

### Understanding AgentCore Runtime Deployment

**AgentCore Runtime** is AWS's managed service for hosting AI agents at scale. When you deploy an agent using the `bedrock_agentcore_starter_toolkit`, several automated processes occur:

#### What Happens During Deployment:

<div style="text-align:left">
    <img src="images/invoke.png" width="75%"/>
</div>

1. **Configuration Phase** (`configure()`): 
   - **Dockerfile Generation**: Creates a containerized environment for your agent
   - **Dependency Management**: Processes your `requirements.txt` for Python dependencies
   - **Execution Role Creation**: Sets up IAM roles with necessary permissions (ECR, CloudWatch, Bedrock)
   - **ECR Repository Setup**: Creates Amazon ECR repository for container images
   - **OpenTelemetry Integration**: Automatically adds observability instrumentation

2. **Build Phase** (`launch()`): 
   - **Container Build**: Packages your agent code into a Docker container
   - **Image Push**: Uploads container to Amazon ECR
   - **Runtime Creation**: Deploys container to AgentCore Runtime infrastructure
   - **Health Checks**: Ensures agent is responsive via `/ping` endpoint

3. **Files Created Automatically**: 
   - `Dockerfile`: Container definition with OpenTelemetry instrumentation
   - `.dockerignore`: Excludes unnecessary files from container build
   - `.bedrock_agentcore.yaml`: Configuration metadata for the deployment

#### Key Benefits:
- **Serverless Scaling**: Automatically scales based on demand
- **Built-in Observability**: OpenTelemetry traces to CloudWatch
- **Security**: IAM-based access control and VPC isolation
- **High Availability**: Multi-AZ deployment with health monitoring

In [ ]:
agent_name = "lab5_strands_agent"
agent_file = str(Path("lab5_agent.py").absolute())

print(f"Configuring agent: {agent_name}")
configure_response = agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=str(Path("requirements.txt").absolute()),
    region=region,
    agent_name=agent_name,
)

print("Deploying to AgentCore...")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print(f"Agent deployed: {launch_result.agent_arn}")
print("\n✅ AgentCore observability automatically enabled:")
print("   - OpenTelemetry instrumentation active")
print("   - Traces will appear in Amazon CloudWatch")
print("   - No additional configuration required")

### AgentCore Runtime Architecture

Once deployed, your agent runs in a managed container environment:


**Default Observability Features:**
- **Automatic Tracing**: Every agent invocation creates traces
- **Performance Metrics**: Response times, error rates, throughput
- **Tool Usage Tracking**: Monitor which tools are called and their performance
- **Model Interaction Logs**: Track LLM requests and responses

In [ ]:
# Invoke lab5_agent remotely using AgentCore API

response = control_client.list_agent_runtimes()
agents = response.get("agentRuntimes", [])
lab5_agent = next((a for a in agents if "lab5_strands_agent" in a["agentRuntimeName"]), None)

if not lab5_agent:
    print("No lab5 agent found. Available agents:")
    for agent in agents:
        print(f"  - {agent['agentRuntimeName']}: {agent['agentRuntimeArn']}")
    exit(1)

agent_arn = lab5_agent["agentRuntimeArn"]
print(f"Found agent: {agent_arn}")

# Example invocation
print("\n=== Test: Basic Agent ===")
response = invoke_agent(agent_arn, "What is 12 multiplied by 15?")
display(Markdown(response))

### AgentCore Observability on Amazon CloudWatch 

To summarize, please follow the below steps to enable observability from AgentCore runtime hosted agents: 

- Enable Transaction Search on Amazon CloudWatch 
- The runtime agent is instrumented using opentelemtry command : `opentelemetry-instrument python any_runtime_agent.py`
- In the Dockefile, notice that it tries to install `aws-opentelemetry-distro` when deploying the agent on Bedrock Agentcore Runtime.


## Bedrock AgentCore Overview on GenAI Observability dashboard 

You are able to view all your Agents that have observability in them and filter the data based on time frames, some examples are provided below :

<div style="text-align:left">
    <img src="images/agentcore_obs_home.png" width="100%"/>
</div>

In the main dashboard you are able to view all model invocations accross all agents as shown below: 

<div style="text-align:left">
    <img src="images/agentcore_obs_model_invocations.png" width="100%"/>
</div>

Now, if you click on the agent you just deployed you will be taken to a dashboard for the runtime metrics specific to this agent, you can also filter the data by a custom time frame.

In the Trace View tab, you can look into the traces and span information for this agent on runtime. 

<div style="text-align:left">
    <img src="images/agentcore_obs_trace.png" width="100%"/>
</div>


Please click through the various features of GenAI observability dashboard to get more detailed information on traces.


## Create, Deploy, and Invoke the Second Agent

Now we'll explore using **Langfuse** instead of AgentCore Observability. Langfuse provides advanced GenAIOps capabilities including detailed trace analysis, cost tracking, and performance analytics specifically designed for LLM applications.

This section will create the second agent file (`lab5_agent_tools.py`), deploy it to AgentCore Runtime, and invoke it remotely.

### Agent 2: Custom Tool Agent (AWS Health Status Checker)
This agent extends the previous setup by introducing a custom tool: `aws_health_status_checker`. It uses the AWS Health Dashboard RSS feed to check the operational status of AWS services in a specified region.

- **Key features:**
  - Defines a custom tool using the `@tool` decorator to fetch and parse AWS Health RSS feed.
  - Responds to user queries about AWS service health, optionally filtered by region and service name.
  - Langfuse tracing is enabled for all agent actions and tool invocations.
  - Deployed as a BedrockAgentCoreApp entrypoint, similar to Agent 1.

#### Agent Details
<div style="float: left; margin-right: 20px;">

|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           | |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>
<br style="clear: both;"/>

This agent demonstrates how to add custom Python tools to your agent and trace their usage for observability.

In [ ]:
%%writefile lab5_agent_tools.py
import base64
import os
import xml.etree.ElementTree as ET
from typing import Dict, List, Optional
import requests
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv
from strands import Agent, tool
from strands.telemetry import StrandsTelemetry
from strands_tools import image_reader

load_dotenv()
langfuse_public_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")
langfuse_host = os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")
LANGFUSE_AUTH = base64.b64encode(
    f"{langfuse_public_key}:{langfuse_secret_key}".encode()
).decode()
os.environ["LANGFUSE_PROJECT_NAME"] = "my_llm_project"
os.environ["DISABLE_ADOT_OBSERVABILITY"] = "true"
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com") + "/api/public/otel"
)
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
for k in [
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_PYTHON_DISTRO",
    "OTEL_RESOURCE_ATTRIBUTES",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_PYTHON_EXCLUDED_URLS",
]:
    os.environ.pop(k, None)
app = BedrockAgentCoreApp()
MODEL = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"


@tool
def aws_health_status_checker(
    region: str = "us-west-2", service_name: Optional[str] = None
) -> Dict:
    try:
        rss_url = "https://status.aws.amazon.com/rss/all.rss"
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        items = root.findall(".//item")
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = (
                item.find("description").text
                if item.find("description") is not None
                else ""
            )
            pub_date = (
                item.find("pubDate").text if item.find("pubDate") is not None else ""
            )
            link = item.find("link").text if item.find("link") is not None else ""
            if region.lower() in title.lower():
                if service_name is None or service_name.lower() in title.lower():
                    events.append(
                        {
                            "title": title,
                            "description": description,
                            "published_date": pub_date,
                            "link": link,
                        }
                    )
        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally"
                if not service_name
                else f"{service_name} in {region} appears to be operating normally",
                "events": [],
            }
        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events,
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": [],
        }


@app.entrypoint
def lab5_agent(payload):
    user_input = payload.get("prompt")
    print("LAB5: User input:", user_input)
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()
    agent = Agent(
        system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
        tools=[aws_health_status_checker],
        model=MODEL,
        name="lab5_strands_agent_custom_tool_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": ["AWS-Strands-Agent", "Custom-Tool"],
            "metadata": {
                "environment": "development",
                "version": "1.0.0",
                "query_type": "storage_recommendation",
            },
        },
    )
    response = agent(user_input)
    response_text = response.message["content"][0]["text"]
    print(f"LAB5: Response preview: {response_text[:100]}...")
    return response_text


if __name__ == "__main__":
    app.run()

### AgentCore Runtime Deployment with Langfuse Integration

This deployment introduces **Langfuse observability** instead of AgentCore's default CloudWatch observability. The key differences in this deployment process:

#### Configuration Changes:

1. **Dockerfile CMD Modification**: 
   - **Default**: `CMD ["opentelemetry-instrument", "python", "-m", "lab5_agent_tools"]`
   - **Modified**: `CMD ["python", "-m", "lab5_agent_tools"]`
   - This disables AgentCore's automatic OpenTelemetry instrumentation

2. **Environment Variables for Langfuse**:
   - `LANGFUSE_HOST`: Langfuse server endpoint
   - `LANGFUSE_PUBLIC_KEY`: Authentication public key
   - `LANGFUSE_SECRET_KEY`: Authentication secret key
   - `DISABLE_ADOT_OBSERVABILITY=true`: Disables AWS Distro for OpenTelemetry

3. **Agent Code Integration**:
   - `StrandsTelemetry().setup_otlp_exporter()`: Configures Langfuse OTLP endpoint
   - Custom trace attributes for enhanced observability
   - Direct OTLP export to Langfuse instead of CloudWatch

#### Observability Approach Considerations:

Both AgentCore's built-in observability and Langfuse integration have their respective advantages and trade-offs. This lab demonstrates Langfuse integration to showcase the flexibility of AgentCore Runtime in supporting different observability solutions.

**Note**: The choice between observability tools depends on your specific requirements, existing infrastructure, and team preferences. Each approach offers different capabilities and integration patterns.

In [ ]:
# Deploy lab5_agent_tools.py to AgentCore Runtime
agent_name = "lab5_strands_agent_custom_tool_example"
agent_file = str(Path("lab5_agent_tools.py").absolute())

print(f"Configuring agent: {agent_name}")
configure_response = agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=str(Path("requirements.txt").absolute()),
    region=region,
    agent_name=agent_name,
)

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    dockerfile_content = dockerfile_path.read_text()
    dockerfile_content = dockerfile_content.replace(
        'CMD ["opentelemetry-instrument", "python", "-m", "lab5_agent_tools"]',
        'CMD ["python", "-m", "lab5_agent_tools"]',
    )
    dockerfile_path.write_text(dockerfile_content)
    print("✅ Dockerfile modified to disable opentelemetry-instrument")

env_vars = {
    "LANGFUSE_HOST": os.environ.get("LANGFUSE_HOST"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
print(f"Agent deployed: {launch_result.agent_arn}")

In [ ]:
# Invoke lab5_agent_tools remotely using AgentCore API
response = control_client.list_agent_runtimes()
agents = response.get("agentRuntimes", [])
lab5_agent_tool = next(
    (a for a in agents if "lab5_strands_agent_custom_tool_example" in a["agentRuntimeName"]),
    None,
)

if not lab5_agent_tool:
    print("No lab5 agent tool found. Available agents:")
    for agent in agents:
        print(f"  - {agent['agentRuntimeName']}: {agent['agentRuntimeArn']}")
    exit(1)

agent_arn = lab5_agent_tool["agentRuntimeArn"]
print(f"Found agent: {agent_arn}")

# Example invocation
print("\n=== Test: AWS Health Tool Agent ===")
response = invoke_agent(
    agent_arn,
    "Check the AWS service health for any disruptions in the us-west-2 region.",
)
display(Markdown(response))

<div style="text-align:left">
    <img src="images/agentcore_langfuse_tool.png" width="100%"/>
</div>

This trace shows an AWS Strands Agent execution that took 4.52s and cost $0.011724.

**Left panel** displays the execution hierarchy: `invoke_agent` contains two `execute_event_loop_cycle` steps, each with `chat` and `execute_tool` operations. The bottom flow diagram visualizes this path.

**Right panel** shows three sections:
- User's request: "Check the AWS service health for any disruptions in the us-west-2 region"
- Assistant's response with tool results and `finish_reason: "end_turn"` (successful completion)
- Metadata including model (Claude Sonnet 4.5), tools used (`aws_health_status_checker`), session ID, and timestamps

**Key metrics**: 2,158 input tokens → 350 output tokens, 4.52s latency, successful tool execution with no errors.

## Create, Deploy, and Invoke the Third Agent

In this final step, we'll combine **Langfuse observability** with **MCP (Model Context Protocol) tools**. This showcases how Langfuse can trace complex multi-tool interactions and provide deep insights into agent orchestration patterns.

This section will create the third agent file (`lab5_agent_mcp.py`), deploy it to AgentCore Runtime, and invoke it remotely.

### Agent 3: MCP Tool Agent (AWS Documentation & Pricing)
This agent showcases advanced orchestration by integrating **third-party MCP (Model Context Protocol) tools** for AWS documentation and pricing queries. It demonstrates how agents can leverage existing MCP servers to answer complex questions.

**Important Note**: This section uses **pre-built third-party MCP servers** (`awslabs.aws-documentation-mcp-server` and `awslabs.aws-pricing-mcp-server`) via `uvx` commands. It does not cover building custom MCP servers from scratch.

**Key Implementation Details:**
- **MCP Client Setup**: Uses `MCPClient` with `stdio_client` to connect to external MCP servers
- **Third-party MCP Servers**: 
  - `awslabs.aws-documentation-mcp-server@latest`: AWS documentation search
  - `awslabs.aws-pricing-mcp-server@latest`: AWS pricing information
- **Tool Discovery**: `mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()` + `aws_health_status_checker`
- **Context Management**: MCP clients initialized within `with` blocks for proper session handling
- **System Prompt Optimization**: Enforces 3-tool limit to prevent excessive MCP calls
- **IAM Requirements**: Custom execution role needed for AWS Pricing API permissions

#### Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>
<br style="clear: both;"/>

This demonstrates enterprise MCP integration patterns using existing community MCP servers rather than custom implementations.

In [ ]:
%%writefile lab5_agent_mcp.py
import base64
import os
import xml.etree.ElementTree as ET
from typing import Dict, List, Optional
import requests
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv
from strands import Agent, tool
from strands.telemetry import StrandsTelemetry
from strands_tools import image_reader
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

load_dotenv()
langfuse_public_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")
langfuse_host = os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")
LANGFUSE_AUTH = base64.b64encode(
    f"{langfuse_public_key}:{langfuse_secret_key}".encode()
).decode()
os.environ["LANGFUSE_PROJECT_NAME"] = "my_llm_project"
os.environ["DISABLE_ADOT_OBSERVABILITY"] = "true"
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com") + "/api/public/otel"
)
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
for k in [
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_PYTHON_DISTRO",
    "OTEL_RESOURCE_ATTRIBUTES",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_PYTHON_EXCLUDED_URLS",
]:
    os.environ.pop(k, None)
app = BedrockAgentCoreApp()
MODEL = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"


@tool
def aws_health_status_checker(
    region: str = "us-west-2", service_name: Optional[str] = None
) -> Dict:
    try:
        rss_url = "https://status.aws.amazon.com/rss/all.rss"
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        items = root.findall(".//item")
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = (
                item.find("description").text
                if item.find("description") is not None
                else ""
            )
            pub_date = (
                item.find("pubDate").text if item.find("pubDate") is not None else ""
            )
            link = item.find("link").text if item.find("link") is not None else ""
            if region.lower() in title.lower():
                if service_name is None or service_name.lower() in title.lower():
                    events.append(
                        {
                            "title": title,
                            "description": description,
                            "published_date": pub_date,
                            "link": link,
                        }
                    )
        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally"
                if not service_name
                else f"{service_name} in {region} appears to be operating normally",
                "events": [],
            }
        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events,
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Error checking AWS service status: {str(e)}",
            "events": [],
        }


@app.entrypoint
def lab5_agent(payload):
    user_input = payload.get("prompt")
    print("LAB5: User input:", user_input)
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()
    aws_docs_mcp = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
            )
        )
    )
    aws_pricing_mcp = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx", args=["awslabs.aws-pricing-mcp-server@latest"]
            )
        )
    )
    with aws_docs_mcp, aws_pricing_mcp:
        mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
        agent = Agent(
            system_prompt="""You are an AWS expert assistant. Follow these rules strictly:
1. TOOL USAGE LIMITS:
    - Maximum 3 tool calls per response
    - For pricing questions: Use get_pricing_service_attributes ONCE, then answer completely
    - For documentation: Use search_documentation ONCE with specific keywords
    - For health checks: Use aws_health_status_checker for service status
    - NEVER make repetitive calls to the same tool
2. RESPONSE STRATEGY:
    - Provide comprehensive answers based on available tool results
    - If first tool call provides sufficient information, answer immediately
    - Do not search for additional information unless critically missing
3. EFFICIENCY REQUIREMENTS:
    - Combine multiple concepts in single tool calls
    - Use your existing knowledge to supplement tool results
    - Stop tool usage once you have enough information to answer
Answer user questions thoroughly but efficiently.""",
            tools=mcp_tools + [aws_health_status_checker],
            model=MODEL,
            name="lab5_strands_agent_mcp_example",
            trace_attributes={
                "session.id": "aws-mcp-agent-demo-session",
                "user.id": "example-user@example.com",
                "langfuse.tags": ["AWS-Strands-Agent", "MCP-Tools"],
                "metadata": {
                    "environment": "development",
                    "version": "1.0.0",
                    "query_type": "aws_assistance",
                },
            },
        )
        response = agent(user_input)
        response_text = response.message["content"][0]["text"]
        print(f"LAB5: Response preview: {response_text[:100]}...")
        return response_text


if __name__ == "__main__":
    app.run()

#### Create a Custom IAM Role for MCP Agent

The MCP agent requires a custom IAM role because it uses the AWS Pricing MCP server (`awslabs.aws-pricing-mcp-server`) which needs specific AWS API permissions that aren't included in the default AgentCore execution role.

**Required AWS Pricing API permissions:**
- `pricing:DescribeServices` - List available AWS services
- `pricing:GetAttributeValues` - Get pricing attributes for services
- `pricing:GetProducts` - Retrieve pricing information for AWS products

Without these permissions, the MCP agent would fail when attempting to query AWS pricing information through the MCP server.

In [ ]:
# Create custom IAM role with AWS Pricing permissions for MCP agent
iam = boto3.client("iam")
sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]

role_name = f"BedrockAgentCore-MCP-ExecutionRole-{region}"
execution_role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"

# Check if role exists, create if not
try:
    iam.get_role(RoleName=role_name)
    print(f"✅ Using existing role: {role_name}")
except iam.exceptions.NoSuchEntityException:
    # Create role with pricing permissions
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
                "Action": "sts:AssumeRole",
            }
        ],
    }

    # Official AgentCore execution role policy with pricing permissions added
    execution_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "ECRImageAccess",
                "Effect": "Allow",
                "Action": ["ecr:BatchGetImage", "ecr:GetDownloadUrlForLayer"],
                "Resource": [f"arn:aws:ecr:{region}:{account_id}:repository/*"],
            },
            {
                "Effect": "Allow",
                "Action": ["logs:DescribeLogStreams", "logs:CreateLogGroup"],
                "Resource": [f"arn:aws:logs:{region}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*"],
            },
            {
                "Effect": "Allow",
                "Action": ["logs:DescribeLogGroups"],
                "Resource": [f"arn:aws:logs:{region}:{account_id}:log-group:*"],
            },
            {
                "Effect": "Allow",
                "Action": ["logs:CreateLogStream", "logs:PutLogEvents"],
                "Resource": [
                    f"arn:aws:logs:{region}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
                ],
            },
            {
                "Sid": "ECRTokenAccess",
                "Effect": "Allow",
                "Action": ["ecr:GetAuthorizationToken"],
                "Resource": "*",
            },
            {
                "Effect": "Allow",
                "Action": [
                    "xray:PutTraceSegments",
                    "xray:PutTelemetryRecords",
                    "xray:GetSamplingRules",
                    "xray:GetSamplingTargets",
                ],
                "Resource": ["*"],
            },
            {
                "Effect": "Allow",
                "Resource": "*",
                "Action": "cloudwatch:PutMetricData",
                "Condition": {"StringEquals": {"cloudwatch:namespace": "bedrock-agentcore"}},
            },
            {
                "Sid": "GetAgentAccessToken",
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:GetWorkloadAccessToken",
                    "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                    "bedrock-agentcore:GetWorkloadAccessTokenForUserId",
                ],
                "Resource": [
                    f"arn:aws:bedrock-agentcore:{region}:{account_id}:workload-identity-directory/default",
                    f"arn:aws:bedrock-agentcore:{region}:{account_id}:workload-identity-directory/default/workload-identity/{agent_name}-*",
                ],
            },
            {
                "Sid": "BedrockModelInvocation",
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream",
                ],
                "Resource": [
                    "arn:aws:bedrock:*::foundation-model/*",
                    f"arn:aws:bedrock:{region}:{account_id}:*",
                ],
            },
            {
                "Sid": "AWSPricingAccess",
                "Effect": "Allow",
                "Action": [
                    "pricing:DescribeServices",
                    "pricing:GetAttributeValues",
                    "pricing:GetProducts",
                ],
                "Resource": "*",
            },
        ],
    }

    # Create role
    iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Execution role for Bedrock AgentCore with AWS Pricing permissions",
    )

    # Attach execution policy with pricing permissions
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="BedrockAgentCoreExecutionPolicy",
        PolicyDocument=json.dumps(execution_policy),
    )

    print(f"✅ Created role with pricing permissions: {role_name}")

In [ ]:
# Deploy lab5_agent_mcp.py to AgentCore Runtime (with custom execution role for pricing permissions)
agent_name = "lab5_strands_agent_mcp_example"
agent_file = str(Path("lab5_agent_mcp.py").absolute())

print(f"Configuring agent: {agent_name}")
configure_response = agentcore_runtime.configure(
    entrypoint=agent_file,
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file=str(Path("requirements.txt").absolute()),
    region=region,
    agent_name=agent_name,
)

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    dockerfile_content = dockerfile_path.read_text()
    dockerfile_content = dockerfile_content.replace(
        'CMD ["opentelemetry-instrument", "python", "-m", "lab5_agent_mcp"]',
        'CMD ["python", "-m", "lab5_agent_mcp"]',
    )
    dockerfile_path.write_text(dockerfile_content)
    print("✅ Dockerfile modified to disable opentelemetry-instrument")

env_vars = {
    "LANGFUSE_HOST": os.environ.get("LANGFUSE_HOST"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
    "PATH": "/usr/local/bin:/usr/bin:/bin",
}

print("Deploying to AgentCore...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
print(f"Agent deployed: {launch_result.agent_arn}")

In [ ]:
# Invoke lab5_agent_mcp remotely using AgentCore API
response = control_client.list_agent_runtimes()
agents = response.get("agentRuntimes", [])
lab5_agent_mcp = next(
    (a for a in agents if "lab5_strands_agent_mcp_example" in a["agentRuntimeName"]),
    None,
)

if not lab5_agent_mcp:
    print("No lab5 MCP agent found. Available agents:")
    for agent in agents:
        print(f"  - {agent['agentRuntimeName']}: {agent['agentRuntimeArn']}")
    exit(1)

agent_arn = lab5_agent_mcp["agentRuntimeArn"]
print(f"Found agent: {agent_arn}")

# Example invocation
print("\n=== Test: MCP Agent ===")
response = invoke_agent(agent_arn, "What is the pricing model for Amazon S3?")
display(Markdown(response))

print("\n=== Test: AWS Health Tool Agent ===")
response = invoke_agent(
    agent_arn,
    "Check the AWS service health for any disruptions in the us-west-2 region.",
)
display(Markdown(response))

<div style="text-align:left">
    <img src="images/agentcore_langfuse_mcp_1.png" width="100%"/>
</div>

<div style="text-align:left">
    <img src="images/agentcore_langfuse_mcp_2.png" width="100%"/>
</div>

**Trace 1: Multi-Tool Execution with MCP**

This trace (23.23s, $0.3576) demonstrates complex agent reasoning across three event loop cycles. When asked about S3 pricing, the agent intelligently decides which MCP-provided tools are needed, first calling `get_pricing_service_attributes` to understand available pricing dimensions, then `get_pricing` to fetch actual rates. MCP (Model Context Protocol) enables the agent to access standardized AWS documentation and pricing servers without custom integration code, providing dynamic, up-to-date information beyond the model's training data. 

Token usage: 109,670 → 1,906 (Σ 111,576). 

The flow diagram shows the agent's multi-step decision-making process with sequential tool executions. The agent synthesizes pricing information across storage tiers, intelligent-tiering options, and request types into a comprehensive structured response.

**Trace 2: Simple Single-Tool Execution**

This trace (5.89s, $0.12888) shows a straightforward agent workflow with two event loop cycles. The agent receives a health check request, autonomously selects the `aws_health_status_checker` tool from its available toolset, executes it once, and returns results. 

Token usage: 41,240 → 344 (Σ 41,584). 

The flow diagram shows linear progression: start → invoke_agent → execute_event_loop_cycle → chat/tool → end. The assistant provides health status and offers follow-up assistance, ending with `finish_reason: "end_turn"`.

**Agent Tool Selection Process**

The agent uses Claude's reasoning capabilities to analyze the user query and autonomously select appropriate tools from its available toolset—both custom tools like `aws_health_status_checker` and MCP-provided tools like `get_pricing_service_attributes`, `get_pricing`, and `search_documentation`. The system prompt guides the agent to use tools efficiently with a 3-tool maximum per response.

## Conclusion: AgentCore Runtime with Observability

As a recap, AgentCore provides a comprehensive platform with these composable components:

- **Runtime**: Secure, serverless capability that empowers organizations to deploy and scale AI agents and tools regardless of framework, protocol, or model choice 
- **Observability**: Helps developers trace, debug, and monitor agent performance through unified operational dashboards. With support for OpenTelemetry compatible telemetry and detailed visualizations of each step of the agent workflow.
- **Gateway**: Automatically converts APIs, Lambda functions, and existing services into MCP-compatible tools so developers can quickly make capabilities available to agents
- **Memory**: Fully-managed memory infrastructure that makes it easy for developers to build rich, personalized agent experiences with customizable memory options
- **Identity**: Provides seamless agent identity and access management across AWS services and third-party applications while supporting standard identity providers
- **Code Interpreter Tool**: Built-in tool that enables AI agents to write and execute code securely, enhancing accuracy and expanding problem-solving capabilities
- **Browser Tool**: Enterprise-grade capability that enables AI agents to navigate websites, complete forms, and perform web-based tasks within a secure sandbox environment


This **AgentCore + Observability** lab completes the GenAIOps workshop series by providing the operational foundation necessary for sustainable AI agent deployments. The integration patterns demonstrated here enable organizations to:

- Monitor Production Agents: Ensure reliability and performance in production environments
- Optimize Costs: Track and manage expenses across agent deployments
- Improve Quality: Detect and resolve issues before they impact users
- Scale Operations: Support growing agent deployments with appropriate observability infrastructure

The combination of AgentCore Runtime's flexibility and multiple observability platform options ensures that organizations can implement GenAIOps practices aligned with their operational requirements and strategic objectives.


## Cleanup: Remove All Agent Resources

Clean up all AgentCore Runtime resources created in this lab to avoid ongoing charges.

In [ ]:
# Clean up all AgentCore Runtime resources
import boto3


control_client = boto3.client("bedrock-agentcore-control")

# List all agent runtimes
response = control_client.list_agent_runtimes()
agents = response.get("agentRuntimes", [])

# Filter lab5 agents
lab5_agents = [a for a in agents if "lab5" in a["agentRuntimeName"].lower()]

if not lab5_agents:
    print("No lab5 agents found to clean up.")
else:
    print(f"Found {len(lab5_agents)} lab5 agents to delete:")
    for agent in lab5_agents:
        print(f"  - {agent['agentRuntimeName']}")

    # Delete each agent
    for agent in lab5_agents:
        try:
            # Extract agent ID from ARN (last part after the last '/')
            agent_id = agent["agentRuntimeArn"].split("/")[-1]
            control_client.delete_agent_runtime(agentRuntimeId=agent_id)
            print(f"✅ Deleted: {agent['agentRuntimeName']}")
        except Exception as e:
            print(f"❌ Failed to delete {agent['agentRuntimeName']}: {e}")

    print("\n🧹 Cleanup completed. All lab5 agents have been removed.")